In [ ]:
import stripy
import meshplex
import numpy as np
import pyvista as pv
from scipy import interpolate
from PlateBoundaries import *
from TectonicEarth import Earth
from gospl._fortran import definegtin
from EarthsAssistant import EarthAssist

# Gospl

Gospl is a python based numerical model for simulating various global scale erosion processes. It takes plate tectonic landscape displacements along with climatic data as input, and simulates *long-term catchment dynamic and drainage evolution as well as sedimentary basins formation*. We will analyze how to run simulations in Gospl to gain a better understanding of it.

### Input yml Files

In Gospl, simulation properties and input files are specified by *.yml* files. We will discuss an example input *.yml* file, and provide examples for generating and reading data referenced by the *.yml* files. For a more detailed documentation of *.yml* attributes used in *Gospl*, see the [official user guide](https://gospl.readthedocs.io/en/latest/user_guide/inputfile.html) documentation.

##### Domain

A *Gospl* input *yml* file begins by specifying the name, and the domain of the erosion simulation. It declares the location of the *.npz* file containing data of the initial surface mesh to be used by *Gospl*, some other simulation parameters, and takes the following form:

``` yml
name: Example input yml file

domain:
  npdata: 'ExampleInput/0Ma'
  flowdir: 5
  fast: False
  backward: False
  interp: 1
 ```
 
The main parameter of our interest here is *npdata*, which specifies the file location of the initial surface mesh to be used by Gospl. The file needs to be an *.npz* file, which essentially multiple numpy arrays stored as a file. The *ExampleInput/0Ma.npz* file was taken from the Gospl *dualLithology* notebook tutorial, and contains the following arrays:

- v: Vertices
- c: Cells
- n: Nearest Neighbours
- z: Elevations

In [ ]:
#Load npz file
npzFile = np.load('ExampleInput/0Ma.npz')
print(npzFile.files)

#Get arrays stored in file
vertices = npzFile['v']
cells = npzFile['c']
nearestNeighbours = npzFile['n']
elevations = npzFile['z']

#Print the shape of arrays within the npz file
print(vertices.shape)
print(cells.shape)
print(nearestNeighbours.shape)
print(elevations.shape)

We can create a pyvista mesh object for visualizations of the earth's initial elevation from the npz file. To do so we will require an array of exagerated earth's radius, and a list of faces, which can be done as follows:

In [ ]:
#Read npz file and return as numpy arrays
def readNPZfile(npzFileDir):
    npzFile = np.load(npzFileDir)
    return npzFile['v'], npzFile['c'], npzFile['n'], npzFile['z']

#Given an npz file, we create an pyvista mesh of earth
def createMeshFromNPZdata(vertices, cells, heights, heightAmplification=30):
    faces = []
    exageratedRadius = vertices + heightAmplification * heights * vertices / np.max(vertices)
    for cell in cells:
        faces.append(3)
        faces.append(cell[0])
        faces.append(cell[1])
        faces.append(cell[2])
    earthMesh = pv.PolyData(exageratedRadius, faces)
    earthMesh['heights'] = heights
    return earthMesh

#Run the newly created functions
vertices, cells, neighbours, heights = readNPZfile('ExampleInput/0Ma.npz')
earthMesh = createMeshFromNPZdata(vertices, cells, heights)
contour = earthMesh.contour([0])

#Display mesh
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(contour, color="black", opacity=1.)
plotter.show()

We would now like to create an appropriate *npz* file from our earth object as discussed in our previous notebooks. Because erosion algorithms used in Gospl depend on nearby neighbours of vertices, it benefits from using an icosahedral sphere (Icosphere), instead of the UV sphere that we have been using so far. So we begin by interpolating heights from our UV sphere to heights on an Icosphere, which is generated by the stripy library.

In [ ]:
#Create an icosphere and interpolate earth heights onto it
def earthToIcosphere(earth, subdivisions=6, iteration=-1):
    icosphere = stripy.spherical_meshes.icosahedral_mesh(
                    include_face_points = False, 
                    refinement_levels = subdivisions)
    icosphereXYZ = icosphere._points * earth.earthRadius
    radLonLat = EarthAssist.cartesianToPolarCoords(icosphereXYZ)
    icoLonLat = np.stack((radLonLat[1], radLonLat[2]), axis=1)
    icoCells = icosphere.simplices
    
    #Interpolate heights
    earthHeights = earth.heightHistory[iteration]
    icoLonLat = np.stack((radLonLat[1], radLonLat[2]), axis=1)
    icoHeights = interpolate.griddata(earth.lonLat, earthHeights, icoLonLat, method='cubic')
    icoHeights = icoHeights[:, np.newaxis]
    return icosphereXYZ, icoCells, icoHeights

The above function takes an *Earth* object as input, and returns arrays suitable for the vertices, cells and elevations that need to be written to the *.npz* file. The only array missing now is the nearestNeighbours array, which is generated by the function bellow which is based on code from the *bfModel* notebook tutorial provided by *Gospl*.

In [ ]:
#Create list of neighbour ids based on bfModel notebook tutorial
def getNeighbourIds(icoXYZ, icoCells):
    Gmesh = meshplex.MeshTri(icoXYZ, icoCells)
    s = Gmesh.idx_hierarchy.shape
    a = np.sort(Gmesh.idx_hierarchy.reshape(s[0], -1).T)
    Gmesh.edges = {'points': np.unique(a, axis=0)}
    ngbNbs, ngbID = definegtin(len(icoXYZ), Gmesh.cells['points'], Gmesh.edges['points'])
    ngbIDs = ngbID[:,:8].astype(int)
    return ngbIDs

#Create an appropriate npz file from earth object and return the file name of the new npz file
def createNPZfromEarth(earth, outDirectory='GeneratedInputFiles/', iteration=-1, subdivisions=6):
    vertices, cells, heights = earthToIcosphere(earth, subdivisions=subdivisions)
    neighbours = getNeighbourIds(vertices, cells)
    
    #Create appropriate file name and save data as npz
    time = earth.timeHistory[iteration]
    fileName = '{}Subdivisions{}Time{}Mya'.format(outDirectory, subdivisions, time)
    np.savez_compressed(fileName, v=vertices, c=cells, n=neighbours.astype(int), z=heights)
    return fileName + '.npz'
    

#Create instance of earth object to run newly created functions with
earth = Earth(
            startTime = 30,
            endTime = 0,
            deltaTime = 5,
            baseUplift = 2000,
            distTransRange = 1000000, 
            numToAverageOver = 10,
            earthRadius = 6378137.,
            useKilometres = False
)

#Create the npz file to be used by Gospl
npzFileName = createNPZfromEarth(earth)

#Read the newly created npz file as before to check that everything is working as expected
vertices, cells, neighbours, heights = readNPZfile(npzFileName)
earthMesh = createMeshFromNPZdata(vertices, cells, heights)
contour = earthMesh.contour([0])

#Display the mesh
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(contour, color="black", opacity=1.)
plotter.show()